In [1]:
'''
    Copyright 2022 by Michał Stolarz <michal.stolarz@h-brs.de>

    This file is part of migrave_personalised_behaviour_model.
    It is used for merging two types of data:
    (i) expected engagement that is calculated in the engagement_extraction (eng_final.csv),
    (ii) user game performance data that was collected from the experiments (perf_final.csv),
    with respect to the timestamps and plotting the full evolution of the interaction during the sequence learning game.

    migrave_personalised_behaviour_model is free software: you can redistribute it and/or modify
    it under the terms of the GNU Affero General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    migrave_personalised_behaviour_model is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Affero General Public License for more details.
    You should have received a copy of the GNU Affero General Public License
    along with migrave_personalised_behaviour_model. If not, see <http://www.gnu.org/licenses/>.
'''

import pandas as pd
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('font', **{'size': 35})
matplotlib.use("pgf")
matplotlib.rcParams.update({
    'axes.labelsize': 100, 'legend.fontsize': 35, 'xtick.labelsize': 35, 'ytick.labelsize': 35,
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
# Lodaing engagement data
eng_df = pd.read_csv("eng_final.csv")
# Loadinging game performance data
perf_df = pd.read_csv("perf_final.csv")
eng_df_group = eng_df.groupby("participant_id")
feedback_shift = 5
plot_users_games = True

# Dictionary with the duration time of saying the sequence (of the specified length) by the robot
sequence_telling_duration_map = {3: 8, 5: 14, 7: 20}
switch_attention_duration = 1

In [3]:
# Exponentially weighted smoothing of the expected engagement data
for key in eng_df_group.groups.keys():
    eng_qt = eng_df_group.get_group(key)['eng_qt'].ewm(alpha=0.75).mean()
    eng_df.loc[eng_df['participant_id']==key, 'eng_qt'] = list(eng_qt)

eng_df.dropna(inplace=True)

In [ ]:
eng_df

In [5]:
# Merging engagement and performance data

# Measure engagement with the time shift when the feedback is given
perf_df_cpy = perf_df.copy(deep=True)
final_df = pd.merge(left=perf_df_cpy, right=eng_df, on=['participant_id', 'secs'])
final_df['feedback'] = final_df['feedback'].shift(1, fill_value=0)
final_df['start'] = 0
final_df['talking_time'] = final_df['length']
final_df = final_df.replace({'talking_time': sequence_telling_duration_map})

for user in final_df['participant_id'].unique():
    final_df.loc[final_df['participant_id']== user, 'start'] = \
        final_df.loc[final_df['participant_id']== user, 'secs'].shift(1, fill_value=0)
    final_df.loc[(final_df['participant_id']== user) &
                 (final_df['start']==0), 'start'] = \
        int(final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'secs'] - \
        final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'duration'] -\
        final_df.loc[(final_df['participant_id']== user) & (final_df['start']==0), 'talking_time'])

# Average engagement over entire task solving time + time when feedback is given
for timestamp in list(final_df['secs']):
    timestamp = int(timestamp)
    down_lim = int(final_df.loc[final_df['secs']==timestamp, 'start'])
    up_lim = timestamp - int(final_df.loc[final_df['secs']==timestamp, 'duration'])

    final_df.loc[final_df['secs']==timestamp, 'eng_qt'] = eng_df.loc[(eng_df['secs'] >= down_lim) & (eng_df['secs'] <= up_lim), 'eng_qt'].mean()

final_df.to_csv("output/final.csv", header=True, index=None, sep=',', float_format='%10.4f', mode='w')

In [ ]:
final_df

In [7]:
# Plotting the evolution of the expected engagement over the entire interaction
if plot_users_games:
    for user in eng_df_group.groups.keys():
        qt = eng_df.groupby("participant_id").get_group(user)[['eng_qt', 'secs']]
        time_offset = qt['secs'].to_numpy()[0]
        qt['secs'] = qt['secs']-time_offset

        length = perf_df.groupby("participant_id").get_group(user)[['length', 'secs']]
        length['secs'] = length['secs']-time_offset

        length_3 = length.loc[length['length'] == 3, 'secs']
        length_3 = np.array([length_3, length_3])

        length_5 = length.loc[length['length'] == 5, 'secs']
        length_5 = np.array([length_5, length_5])

        length_7 = length.loc[length['length'] == 7, 'secs']
        length_7 = np.array([length_7, length_7])

        solving_start = perf_df.groupby('participant_id').get_group(user)[['duration', 'secs']].round()
        solving_start['secs'] = solving_start['secs']-time_offset
        solving_start = solving_start['secs'] - solving_start['duration']

        feedback = perf_df.groupby("participant_id").get_group(user)[['feedback', 'secs']]
        feedback['secs'] = feedback['secs']-time_offset
        feedback_en = feedback.loc[feedback['feedback']==1, 'secs']+feedback_shift
        feedback_ch = feedback.loc[feedback['feedback']==2, 'secs']+feedback_shift

        plt.figure(figsize=(20, 4), dpi=100)
        plt.rc('axes', labelsize=100)
        qt_handl, = plt.plot(qt.secs, qt.eng_qt, label="Engagement", color='b')
        #tab_handl, = plt.plot(tab.secs, tab.eng_tab, label="Engagement from tablet", color='c')
        length_3_handl = plt.plot(length_3, np.array([[-1, 1]]*length_3.shape[1]).T, label="End of level 1", color='r', linestyle='-', linewidth=3)
        length_5_handl = plt.plot(length_5, np.array([[-1, 1]]*length_5.shape[1]).T, label="End of level 2", color='r', linestyle='--', linewidth=3)
        length_7_handl = plt.plot(length_7, np.array([[-1, 1]]*length_7.shape[1]).T, label="End of level 3", color='r', linestyle=':', linewidth=3)

        solv_start_handl = plt.scatter(solving_start, [0]*solving_start.shape[0], label="Solving start", color='black', linewidths=20, marker='x')

        feedback_en_handl = plt.scatter(feedback_en, [0]*feedback_en.shape[0], label="Encouraging feedback", color='m', linewidths=20, marker='o')
        feedback_ch_handl = plt.scatter(feedback_ch, [0]*feedback_ch.shape[0], label="Challenging feedback", color='y', linewidths=20, marker='o')

        plt.legend(handles=[qt_handl, length_3_handl[0], length_5_handl[0], length_7_handl[0], solv_start_handl, feedback_en_handl, feedback_ch_handl], bbox_to_anchor=(1.33, 0.5), loc='center right')

        # Plots for the report
        if user in ['J0YH72SI', '5J7PWO3G']:
            plt.ylabel("Engagement")
            plt.xlabel("Time [s]")
            if user=='J0YH72SI':
                plt.xlim(50, 435)
            elif user == '5J7PWO3G':
                plt.xlim(50, 470)
            plt.grid()
            plt.savefig(f"plots/{user}_sequence_learning.pdf", bbox_inches='tight')
        else:
            plt.ylabel("Engagement")
            plt.xlabel("Time [s]")
            plt.grid()
            plt.title(f"Sequence learning game for user {user}")
            plt.savefig(f"plots/{user}_sequence_learning.pdf", bbox_inches='tight')